In [1]:
import pandas as pd

# Librería para conexión a base de datos
import mysql.connector

# Librería para procesamiento de lenguaje natural(NLP)
import spacy  
from textblob import TextBlob


**Observaciones:** <br><br>
Se decidió guardar los datos en una base de datos MySQL, ya que al contar con una gran cantidad de registros, el procesamiento de los mismos se vuelve muy lento y no fue posible obtener el dataframe completo.


In [2]:
# Conexión con la base de datos
cnx = mysql.connector.connect(host='localhost',
                              port = '3306',
                              database='corpus_linguistico',
                              user='root',
                              password='')

In [3]:
# Instancia de cursor
cursor = cnx.cursor()

## Agregar nuevas variables al dataset

### Análisis de sentimiento

Para realizar el ánalisis de sentimiento a cada frase se utilizó la librería **"TextBlob"**

**Textblob:** librería de procesamiento de lenguaje natural en Python que se utiliza para tareas como análisis de sentimientos, extracción de información, clasificación de textos, traducción, entre otras.

In [4]:
# Función para clasificar el sentimiento de una frase
    # Argumentos: frase
    # Salida: sentimiento

def analiza_sentimiento(frase):
    testimonio = TextBlob(frase)                    # Crea un objeto TextBlob con la frase 
    polaridad = testimonio.sentiment.polarity       # Extrae la polaridad del sentimiento de la frase 
    if polaridad > 0:
        return "positivo"
    elif polaridad < 0:
        return "negativo"
    else:
        return "neutral"

### Categorías gramaticales

Para identificar las categorías presentes en cada frase se utilizó la librería "Spacy"

**Spacy**: es una biblioteca de procesamiento de lenguaje natural muy completa y eficiente, es muy útil para el procesamiento de texto, análisis de texto, etiquetado POS.

Las nuevas variables agregar son:
- NOUN: Sustantivos
- ADJ: Adjetivos
- DET: Determinantes (artículos)
- ADV: Adverbios
- CCONJ: Conjunciones
- VERB: Verbos
- AUX: Verbos auxiliares
- PRON: Pronombres
- ADP: Preposiciones
- PUNCT: signos de puntuación

In [5]:
# Carga el modelo en inglés de Spacy
nlp = spacy.load('en_core_web_lg')      

# Carga el modelo en español de Spacy               
nlpe = spacy.load('es_core_news_lg')  

#### Identificar las partes de cada frase 

In [6]:
# Función para contar palabras de acuerdo al tipo de token (sustantivo, adjetivo, etc)
    # Argumentos: frase, modelo, tipo
                    # modelo: en (inglés) o es (español)
                    # tipo: sustantivo, adjetivo, verbo, etc (tokens de Spacy)

    # Salida: cantidad de palabras del tipo especificado

def contar(frase, modelo, tipo):
    modelo = modelo.lower()         # Convierte a minúsculas el modelo
    if modelo == "en":
        doc = nlp(frase)
    if modelo == "es":
        doc = nlpe(frase)                
    count = 0               
    for token in doc:
        if(token.pos_ == tipo):     # De acuerdo al token, incrementa el contador de ese tipo de palabra
            count += 1
    return count

In [7]:
# Agrega a la base de datos los resultados de la clasificación de sentimiento y conteo de palabras por tipo de token.

while True:

    # Selecciona las 1000 primeras de la tabla tu que la columna sentiment sea null
    query = "SELECT id, source, target FROM tu WHERE sentiment IS NULL AND busy=false LIMIT 1000"

    # Crea un dataframe con los datos de la consulta
    df_sql = pd.read_sql(query, cnx)

    # Identifica los ids de los registros a actualizar y pone busy=true para que no sean seleccionados por otro proceso 
    ids = list(df_sql["id"])  
    qq= "UPDATE tu SET busy=true WHERE id IN (" + ",".join(map(str, ids)) + ")"


    # Actualiza los registros en la tabla
    cursor.execute(qq)
    cnx.commit()

    # Aplica la función analiza_sentimiento a cada fila del dataframe y crea una nueva columna con el resultado de la función
    df_sql['sentiment'] = df_sql['source'].apply(analiza_sentimiento)

    # Aplique la función contar a cada fila del dataframe y cree una nueva columna con el resultado de la función

    # CANTIDAD DE SUSTANTIVOS EN SOURCE Y TARGET
    df_sql["NOUN_source"] = df_sql["source"].apply(contar, args=("en", "NOUN",))
    df_sql["NOUN_target"] = df_sql["target"].apply(contar, args=("es", "NOUN",))

    # CANTIDAD DE ADJETIVOS EN SOURCE Y TARGET
    df_sql["ADJ_source"] = df_sql["source"].apply(contar, args=("en", "ADJ",))
    df_sql["ADJ_target"] = df_sql["target"].apply(contar, args=("es", "ADJ",))

    # CANTIDAD DE DETERMINANTES EN SOURCE Y TARGET
    df_sql["DET_source"] = df_sql["source"].apply(contar, args=("en", "DET",))
    df_sql["DET_target"] = df_sql["target"].apply(contar, args=("es", "DET",))

    # CANTIDAD DE ADVERBIOS EN SOURCE Y TARGET
    df_sql["ADV_source"] = df_sql["source"].apply(contar, args=("en", "ADV",))
    df_sql["ADV_target"] = df_sql["target"].apply(contar, args=("es", "ADV",))

    # CANTIDAD DE CONJUNCIONES EN SOURCE Y TARGET
    df_sql["CCONJ_source"] = df_sql["source"].apply(contar, args=("en", "CCONJ",))
    df_sql["CCONJ_target"] = df_sql["target"].apply(contar, args=("es", "CCONJ",))

    # CANTIDAD DE VERBOS EN SOURCE Y TARGET
    df_sql["VERB_source"] = df_sql["source"].apply(contar, args=("en", "VERB",))
    df_sql["VERB_target"] = df_sql["target"].apply(contar, args=("es", "VERB",))

    # CANTIDAD DE VERBOS AUXILIARES EN SOURCE Y TARGET (do, have, be)
    df_sql["VAUX_source"] = df_sql["source"].apply(contar, args=("en", "AUX",))
    df_sql["VAUX_target"] = df_sql["target"].apply(contar, args=("es", "AUX",))

    # CANTIDAD DE PRONOMBRES EN SOURCE Y TARGET
    df_sql["PRON_source"] = df_sql["source"].apply(contar, args=("en", "PRON",))
    df_sql["PRON_target"] = df_sql["target"].apply(contar, args=("es", "PRON",))

    # CANTIDAD DE PRE\POS POSICIONES EN SOURCE Y TARGET
    df_sql["ADP_source"] = df_sql["source"].apply(contar, args=("en", "ADP",))
    df_sql["ADP_target"] = df_sql["target"].apply(contar, args=("es", "ADP",))

    # CANTIDAD DE SIGNOS DE PUNTUACION EN SOURCE Y TARGET
    df_sql["PUNCT_source"] = df_sql["source"].apply(contar, args=("en", "PUNCT",))
    df_sql["PUNCT_target"] = df_sql["target"].apply(contar, args=("es", "PUNCT",))

    # Selecciona las columnas que se van a actualizar en la tabla tu
    df_sql = df_sql[['sentiment', 'NOUN_source', 'NOUN_target', 'ADJ_source', 'ADJ_target',
                     'DET_source', 'DET_target', 'ADV_source', 'ADV_target', 'CCONJ_source', 'CCONJ_target',
                     'VERB_source', 'VERB_target', 'VAUX_source', 'VAUX_target', 'PRON_source', 'PRON_target',
                     'ADP_source', 'ADP_target', 'PUNCT_source', 'PUNCT_target', 
                        'id']] 
                    

    # Convierte el dataframe en una lista
    values = df_sql.values.tolist()
    if len(values) == 0:
        break

    # Actualiza la tabla tu con los nuevos valores de la columna sentiment y los nuevos valores de las columnas de conteo de palabras por tipo de token

    query = """ UPDATE tu SET sentiment = %s,
                                NOUN_source = %s, NOUN_target = %s, ADJ_source = %s, ADJ_target = %s, 
                                DET_source = %s, DET_target = %s, ADV_source = %s, ADV_target = %s,
                                CCONJ_source = %s,CCONJ_target = %s, VERB_source = %s,VERB_target = %s,
                                VAUX_source = %s,VAUX_target = %s,PRON_source = %s,PRON_target = %s,
                                ADP_source = %s,ADP_target= %s,PUNCT_source = %s,PUNCT_target= %s 
                            WHERE id = %s
    """

    # Ejecución de la actualización en lote
    cursor.executemany(query, values)

    # Confirmar los cambios en la base de datos
    cnx.commit()

    # Imprime el contador de registros actualizados y el número de registros en el dataframe, para verificar que se esté actualizando la tabla
    print(cursor.rowcount, "registros actualizados.")


/var/folders/j7/f0dlbvv5239_cpgr1x6831pc0000gn/T/ipykernel_26410/3903105507.py:7: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_sql = pd.read_sql(query, cnx)


1000 registros actualizados.


/var/folders/j7/f0dlbvv5239_cpgr1x6831pc0000gn/T/ipykernel_26410/3903105507.py:7: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_sql = pd.read_sql(query, cnx)


0 registros actualizados.


/var/folders/j7/f0dlbvv5239_cpgr1x6831pc0000gn/T/ipykernel_26410/3903105507.py:7: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_sql = pd.read_sql(query, cnx)


...

/var/folders/j7/f0dlbvv5239_cpgr1x6831pc0000gn/T/ipykernel_26410/3903105507.py:7: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_sql = pd.read_sql(query, cnx)


0 registros actualizados.


/var/folders/j7/f0dlbvv5239_cpgr1x6831pc0000gn/T/ipykernel_26410/3903105507.py:7: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_sql = pd.read_sql(query, cnx)


0 registros actualizados.


/var/folders/j7/f0dlbvv5239_cpgr1x6831pc0000gn/T/ipykernel_26410/3903105507.py:7: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_sql = pd.read_sql(query, cnx)


[Siguiente: Limpieza de Datos](4_limpieza_datos.ipynb)